In [1]:
!pip install lightgbm==3.3.2
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

You should consider upgrading via the 'C:\Users\user\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.
'sudo'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'sudo'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [2]:
import lightgbm
lightgbm.__version__

'3.3.2'

In [17]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
df = pd.read_csv('data/Final/20230911_인코딩완료.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5484 entries, 0 to 5483
Data columns (total 58 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   매출                      5484 non-null   float64
 1   기준_년_코드                 5484 non-null   int64  
 2   상권_구분_코드_명              5484 non-null   object 
 3   상권_코드                   5484 non-null   int64  
 4   상권_코드_명                 5484 non-null   object 
 5   점포수                     5484 non-null   int64  
 6   시간대1                    5484 non-null   int64  
 7   시간대2                    5484 non-null   int64  
 8   시간대3                    5484 non-null   int64  
 9   시간대4                    5484 non-null   int64  
 10  시간대5                    5484 non-null   int64  
 11  분기_1                    5484 non-null   int64  
 12  분기_2                    5484 non-null   int64  
 13  분기_3                    5484 non-null   int64  
 14  총 상주인구 수                5484 non-null   

In [4]:
filtered_df = df[[
    'area', '점포수', '총 상주인구 수', '총 가구 수', '총_직장인구_수', '아파트_단지_수',
    '아파트_가격_1_억_미만_세대_수', '아파트_가격_1_억_세대_수', '아파트_가격_2_억_세대_수',
    '아파트_가격_3_억_세대_수', '아파트_가격_4_억_세대_수', '아파트_가격_5_억_세대_수',
    '아파트_가격_6_억_이상_세대_수', '총_생활인구_수', '시간대_생활인구_수',
    '월요일_생활인구_수', '화요일_생활인구_수', '수요일_생활인구_수', '목요일_생활인구_수',
    '금요일_생활인구_수', '토요일_생활인구_수', '일요일_생활인구_수', '집객시설_수',
    '관공서_수', '은행_수', '백화점_수', '숙박_시설_수', '연령대_10_생활인구_수',
    '연령대_20_생활인구_수', '연령대_30_생활인구_수', '연령대_40_생활인구_수',
    '연령대_50_생활인구_수', '연령대_60_이상_생활인구_수', '배후지_아파트_단지_수',
    '배후지_아파트_가격_1_억_미만_세대_수', '배후지_아파트_가격_1_억_세대_수',
    '배후지_아파트_가격_2_억_세대_수', '배후지_아파트_가격_3_억_세대_수',
    '배후지_아파트_가격_4_억_세대_수', '배후지_아파트_가격_5_억_세대_수',
    '배후지_아파트_가격_6_억_이상_세대_수', '시간대_버스_승하차승객수',
    '시간대_지하철_승하차승객수', '버스정류장_수', '지하철역_수'
]]
filtered_df

,area,점포수,총 상주인구 수,총 가구 수,총_직장인구_수,아파트_단지_수,아파트_가격_1_억_미만_세대_수,아파트_가격_1_억_세대_수,아파트_가격_2_억_세대_수,아파트_가격_3_억_세대_수,...,배후지_아파트_가격_1_억_세대_수,배후지_아파트_가격_2_억_세대_수,배후지_아파트_가격_3_억_세대_수,배후지_아파트_가격_4_억_세대_수,배후지_아파트_가격_5_억_세대_수,배후지_아파트_가격_6_억_이상_세대_수,시간대_버스_승하차승객수,시간대_지하철_승하차승객수,버스정류장_수,지하철역_수
0,62126,6,618,300,1683,5,0,5,17,8,...,259.0,163.0,64.0,52.0,42.0,1802.0,0,0,0,0
1,29000,3,1039,580,273,16,5,76,41,9,...,436.0,309.0,119.0,81.0,68.0,109.0,0,0,0,0
2,21629,3,879,583,117,27,1,131,89,23,...,780.0,964.0,124.0,151.0,82.0,138.0,0,0,0,0
3,39798,3,497,335,636,3,0,28,2,3,...,285.0,226.0,67.0,49.0,50.0,68.0,0,0,0,0
4,190912,30,7080,4369,3430,175,41,975,495,132,...,2312.0,1553.0,325.0,262.0,204.0,286.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5479,61589,3,1777,797,494,56,0,58,99,91,...,58.0,99.0,91.0,42.0,12.0,1372.0,0,0,0,0
5480,360433,39,4780,3339,48780,93,19,556,843,113,...,NaN,NaN,NaN,NaN,NaN,NaN,125560,907321,6,1
5481,251745,16,1129,598,8579,20,1,53,30,14,...,NaN,NaN,NaN,NaN,NaN,NaN,154697,460651,6,1
5482,256687,15,3152,1777,6518,62,0,154,140,109,...,NaN,NaN,NaN,NaN,NaN,NaN,87513,0,2,0


In [5]:
import pandas as pd

def modify_df(df):
    modified_df = df.copy()  # 원본 데이터프레임을 변경하지 않기 위해 복사본을 만듭니다.
    for col in df.columns[1:]:  # 첫 번째 열(area)은 변환하지 않습니다.
        modified_df[col] = round(df[col] / df['area'] * 10000, 10)
    return modified_df

In [6]:
filtered_df2 = modify_df(filtered_df)
filtered_df2

,area,점포수,총 상주인구 수,총 가구 수,총_직장인구_수,아파트_단지_수,아파트_가격_1_억_미만_세대_수,아파트_가격_1_억_세대_수,아파트_가격_2_억_세대_수,아파트_가격_3_억_세대_수,...,배후지_아파트_가격_1_억_세대_수,배후지_아파트_가격_2_억_세대_수,배후지_아파트_가격_3_억_세대_수,배후지_아파트_가격_4_억_세대_수,배후지_아파트_가격_5_억_세대_수,배후지_아파트_가격_6_억_이상_세대_수,시간대_버스_승하차승객수,시간대_지하철_승하차승객수,버스정류장_수,지하철역_수
0,62126,0.965779,99.475260,48.288961,270.901072,0.804816,0.000000,0.804816,2.736374,1.287706,...,41.689470,26.237002,10.301645,8.370087,6.760455,290.055693,0.000000,0.000000,0.000000,0.000000
1,29000,1.034483,358.275862,200.000000,94.137931,5.517241,1.724138,26.206897,14.137931,3.103448,...,150.344828,106.551724,41.034483,27.931034,23.448276,37.586207,0.000000,0.000000,0.000000,0.000000
2,21629,1.387027,406.398816,269.545518,54.094040,12.483240,0.462342,60.566832,41.148458,10.633871,...,360.626936,445.697906,57.330436,69.813676,37.912063,63.803227,0.000000,0.000000,0.000000,0.000000
3,39798,0.753807,124.880647,84.175084,159.807025,0.753807,0.000000,7.035529,0.502538,0.753807,...,71.611639,56.786773,16.835017,12.312176,12.563445,17.086286,0.000000,0.000000,0.000000,0.000000
4,190912,1.571405,370.851492,228.848894,179.663929,9.166527,2.147586,51.070650,25.928176,6.914180,...,121.102917,81.346379,17.023550,13.723600,10.685551,14.980724,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5479,61589,0.487100,288.525548,129.406225,80.209128,9.092533,0.000000,9.417266,16.074299,14.775366,...,9.417266,16.074299,14.775366,6.819400,1.948400,222.767053,0.000000,0.000000,0.000000,0.000000
5480,360433,1.082032,132.618267,92.638576,1353.372194,2.580230,0.527144,15.425891,23.388535,3.135118,...,NaN,NaN,NaN,NaN,NaN,NaN,3483.587796,25173.083486,0.166466,0.027744
5481,251745,0.635564,44.846968,23.754196,340.781346,0.794455,0.039723,2.105305,1.191682,0.556118,...,NaN,NaN,NaN,NaN,NaN,NaN,6144.987984,18298.317742,0.238336,0.039723
5482,256687,0.584369,122.795467,69.228282,253.927936,2.415393,0.000000,5.999525,5.454113,4.246417,...,NaN,NaN,NaN,NaN,NaN,NaN,3409.327313,0.000000,0.077916,0.000000


In [7]:
selected_df1 = df[['매출', '기준_년_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '시간대1', '시간대2', '시간대3', '시간대4', '시간대5', '분기_1', '분기_2', '분기_3']]
selected_df1

,매출,기준_년_코드,상권_구분_코드_명,상권_코드,상권_코드_명,시간대1,시간대2,시간대3,시간대4,시간대5,분기_1,분기_2,분기_3
0,125163.15020,2020,골목상권,2110949,한남IC,1,0,0,0,0,1,0,0
1,94688.22344,2020,골목상권,2110951,신사은행나무공원,1,0,0,0,0,1,0,0
2,177736.46150,2020,골목상권,2110952,논현목련공원,1,0,0,0,0,1,0,0
3,34587.75458,2020,골목상권,2110954,국기원,1,0,0,0,0,1,0,0
4,221061.48320,2020,골목상권,2110955,논현초등학교,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5479,201683.59780,2022,골목상권,2110994,개포고등학교,0,0,0,0,0,0,0,0
5480,468795.99920,2022,발달상권,2120210,선릉역,0,0,0,0,0,0,0,0
5481,244665.79960,2022,발달상권,2120188,압구정역,0,0,0,0,0,0,0,0
5482,259723.91590,2022,발달상권,2120186,가로수길,0,0,0,0,0,0,0,0


In [8]:
merged_df1 = pd.concat([selected_df1, filtered_df2], axis=1)
merged_df1

,매출,기준_년_코드,상권_구분_코드_명,상권_코드,상권_코드_명,시간대1,시간대2,시간대3,시간대4,시간대5,...,배후지_아파트_가격_1_억_세대_수,배후지_아파트_가격_2_억_세대_수,배후지_아파트_가격_3_억_세대_수,배후지_아파트_가격_4_억_세대_수,배후지_아파트_가격_5_억_세대_수,배후지_아파트_가격_6_억_이상_세대_수,시간대_버스_승하차승객수,시간대_지하철_승하차승객수,버스정류장_수,지하철역_수
0,125163.15020,2020,골목상권,2110949,한남IC,1,0,0,0,0,...,41.689470,26.237002,10.301645,8.370087,6.760455,290.055693,0.000000,0.000000,0.000000,0.000000
1,94688.22344,2020,골목상권,2110951,신사은행나무공원,1,0,0,0,0,...,150.344828,106.551724,41.034483,27.931034,23.448276,37.586207,0.000000,0.000000,0.000000,0.000000
2,177736.46150,2020,골목상권,2110952,논현목련공원,1,0,0,0,0,...,360.626936,445.697906,57.330436,69.813676,37.912063,63.803227,0.000000,0.000000,0.000000,0.000000
3,34587.75458,2020,골목상권,2110954,국기원,1,0,0,0,0,...,71.611639,56.786773,16.835017,12.312176,12.563445,17.086286,0.000000,0.000000,0.000000,0.000000
4,221061.48320,2020,골목상권,2110955,논현초등학교,1,0,0,0,0,...,121.102917,81.346379,17.023550,13.723600,10.685551,14.980724,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5479,201683.59780,2022,골목상권,2110994,개포고등학교,0,0,0,0,0,...,9.417266,16.074299,14.775366,6.819400,1.948400,222.767053,0.000000,0.000000,0.000000,0.000000
5480,468795.99920,2022,발달상권,2120210,선릉역,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,3483.587796,25173.083486,0.166466,0.027744
5481,244665.79960,2022,발달상권,2120188,압구정역,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,6144.987984,18298.317742,0.238336,0.039723
5482,259723.91590,2022,발달상권,2120186,가로수길,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,3409.327313,0.000000,0.077916,0.000000


In [9]:
# merged_df1.to_csv('div_area(sig).csv', index=False, encoding = 'utf-8 sig')